In [ ]:
!pip install pfhedge

In [ ]:
import torch

### Starting example

In [ ]:
from pfhedge.instruments import BrownianStock
from pfhedge.instruments import EuropeanOption

stock = BrownianStock(cost=0.0001)
derivative = EuropeanOption(stock)

derivative

EuropeanOption(
  strike=1., maturity=0.0800
  (underlier): BrownianStock(sigma=0.2000, cost=1.0000e-04, dt=0.0040)
)

### MLP

In [ ]:
from pfhedge.nn import Hedger
from pfhedge.nn import MultiLayerPerceptron

model = MultiLayerPerceptron()
hedger = Hedger(model, inputs=["log_moneyness", "expiry_time", "volatility", "prev_hedge"])

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [ ]:
hedger

Hedger(
  inputs=['log_moneyness', 'expiry_time', 'volatility', 'prev_hedge']
  (model): MultiLayerPerceptron(
    (0): LazyLinear(in_features=0, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=32, bias=True)
    (7): ReLU()
    (8): Linear(in_features=32, out_features=1, bias=True)
    (9): Identity()
  )
  (criterion): EntropicRiskMeasure()
)

In [ ]:
_ = hedger.fit(derivative, n_epochs=200)

Loss=0.0231: 100%|██████████| 200/200 [00:15<00:00, 12.84it/s]


In [ ]:
price = hedger.price(derivative)
price

tensor(0.0215)

### Black-Scholes Model

In [ ]:
from pfhedge.nn import BlackScholes
from pfhedge.nn import Hedger

derivative = EuropeanOption(BrownianStock(cost=1e-4))

model = BlackScholes(derivative)
hedger = Hedger(model, inputs=model.inputs())

In [ ]:
price = hedger.price(derivative)
price

tensor(0.0227)

### Black-Scholes European Options

In [ ]:
from pfhedge.nn import BSEuropeanOption

model = BSEuropeanOption()

In [ ]:
model.inputs()

['log_moneyness', 'time_to_maturity', 'volatility']

In [ ]:
input = torch.tensor([
    [-0.01, 0.1, 0.2],
    [0.0, 0.08, 0.2],
    [0.01, 0.1, 0.2],
])

In [ ]:
model(input)

tensor([[0.4497],
        [0.5113],
        [0.5752]])

In [ ]:
model(input)

tensor([[-0.5503],
        [-0.4887],
        [-0.4248]])